In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
url = 'https://www.baseball-reference.com/leagues/majors/2019-standard-batting.shtml'
r = requests.get(url)
print(r.text)

<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no-rep

In [3]:
bb_soup = BeautifulSoup(r.text, 'html.parser')
print(bb_soup.head())

[<title>Just a moment...</title>, <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>, <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>, <meta content="noindex,nofollow" name="robots"/>, <meta content="width=device-width,initial-scale=1" name="viewport"/>, <style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no-repeat;background-size:conta

In [4]:

# container = bb_soup.find("div", id="switcher_players_standard_batting")

# if not container:
#     print("Container not found!")
# else:
#     table = container.find("table", class_="stats_table")

# df = pd.read_html(str(table), flavor="bs4")[0]
# table = bb_soup.find("table", id="players_standard_batting")
# hitters = []

# hitters = table.find_all("tr")[:1] if table else []

# for hitter in hitters:
#     row = {}
#     hitter_name = hitter.find("td","a", data_stat="name_display")
#     if hitter_name:
#         row['name'] = hitter_name.get_text(strip=True)
# df = pd.DataFrame(hitters)
# print(df)


# Example: scraping the 2019 batting leaderboard (you can change year)


headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"Blocked or page not found for {year}")
    return pd.DataFrame()

soup = BeautifulSoup(response.text, 'html.parser')

# NEW WAY – the table is inside a div with id that contains "players_standard_batting"
container = soup.find('div', id=lambda x: x and 'players_standard_batting' in x)
if not container:
    print(f"Could not find table container for {year}")
    return pd.DataFrame()

table = container.find('table')
if not table:
    print("No table inside container")
    return pd.DataFrame()

rows = table.find('tbody').find_all('tr', class_=lambda x: x != 'thead')  # skip header rows

data = []
salary_cache = {}   # {player_id: {year: salary}} – one cache for the whole run

def get_salary(player_link, target_year):
    player_id = player_link.split('/')[-1].replace('.shtml', '')
    if player_id in salary_cache:
        return salary_cache[player_id].get(target_year, 0)
    
    try:
        full_url = "https://www.baseball-reference.com" + player_link
        page = requests.get(full_url, headers=headers)
        page.raise_for_status()
        p_soup = BeautifulSoup(page.text, 'html.parser')
        
        sal_table = p_soup.find('table', id='br-salaries')
        if not sal_table:
            salary_cache[player_id] = {}
            return 0
        
        salaries = {}
        for tr in sal_table.find('tbody').find_all('tr'):
            cells = tr.find_all(['td', 'th'])
            if len(cells) < 4: continue
            y_text = cells[0].get_text(strip=True)
            if not y_text.isdigit(): continue
            y = int(y_text)
            if y != target_year: continue
            amount = cells[3].get('data-amount')
            if amount and not cells[3].get('data-future'):
                salaries[y] = int(float(amount))
        
        salary_cache[player_id] = salaries
        time.sleep(1.5 + random.uniform(0, 1))  # be nice
        return salaries.get(target_year, 0)
        
    except Exception as e:
        print(f"Salary error {player_link}: {e}")
        salary_cache[player_id] = {}
        return 0

for row in rows:
    name_cell = row.find('td', {'data-stat': 'player'})
    if not name_cell:
        continue
    
    a_tag = name_cell.find('a')
    if not a_tag:
        continue
        
    player_name = a_tag.get_text(strip=True)
    player_link = a_tag['href']                     # e.g. /players/s/semiema01.shtml
    player_id   = name_cell.get('data-append-csv')   # semiema01 – never fails
    
    salary = get_salary(player_link, year)
    
    data.append({
        'Year': year,
        'Player': player_name,
        'Player_ID': player_id,
        'Player_Link': "https://www.baseball-reference.com" + player_link,
        'Salary': salary,
        'Salary_Formatted': f"${salary:,}" if salary > 0 else "$0"
    })

return pd.DataFrame(data)

# ------------------- RUN IT -------------------
years = range(2018, 2026)   # 2018–2025
all_dfs = []

for y in years:
print(f"Scraping {y}...")
df_year = scrape_year_with_salaries(y)
if not df_year.empty:
    all_dfs.append(df_year)
time.sleep(3)   # extra pause between years

final_df = pd.concat(all_dfs, ignore_index=True)
final_df.to_csv("MLB_2018_2025_with_Salaries.csv", index=False)

print("\nFinished!")
print(f"Total rows: {len(final_df)}")
print("\n2025 Top 10 highest paid:")
print(final_df[final_df['Year']==2025][['Player','Salary_Formatted']].nlargest(10, 'Salary'))

IndentationError: expected an indented block after 'for' statement on line 120 (2180305040.py, line 121)